<a href="https://colab.research.google.com/github/aditiiilaturkar/911Interactions/blob/main/hugging_face_svm_naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_ner_bc5cdr_md-0.5.1.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.1
    Uninstalling wasabi-1.1.1:
      Successfully uninstalled wasabi-1.1.1
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.2
    Uninstalling spacy-3.5.2:
      Successfully uninstalled spacy-3.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are insta

In [ ]:
import scispacy
import spacy
import joblib
import re
import string

import numpy as np
import pandas as pd

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, classification_report
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.naive_bayes import MultinomialNB

In [ ]:
import re
from nltk import word_tokenize
from collections import Counter
from tqdm import tqdm
from collections import defaultdict

In [ ]:
data=pd.read_csv("filtered_6months_Whi.csv")  #whisper file

In [ ]:
model = spacy.load("en_ner_bc5cdr_md")


In [ ]:
df = pd.DataFrame(
    dict(
        text=data["TEXT"],
        target=data["nature"].str.lower()))

In [ ]:
df

,text,target
0,"0012.5s 0014.7s SPEAKER_01: 911, what's the a...",fire-commercial
1,"0011.1s 0012.9s SPEAKER_01: 911, let's addres...",sick person
2,"0011.0s 0014.2s SPEAKER_00: 911, what's he ad...",lift assist
3,0004.8s 0006.0s SPEAKER_00: Don't forget to t...,injury (basic)
4,"0010.6s 0012.4s SPEAKER_00: 911, what's the a...",sick person
...,...,...
725,"0011.4s 0013.8s SPEAKER_02: 911, what is the ...",911 cellular hang-up/open/misdial
726,"\n 0014.1s 0018.3s SPEAKER_01: 911, what's th...",fire - lightning no smoke/fire
727,0002.0s 0002.7s SPEAKER_00: What's your phone...,missing person
728,"0011.4s 0013.8s SPEAKER_02: 911, what is the ...",911 cellular hang-up/open/misdial


In [ ]:
df['text'][0]

"0012.5s 0014.7s SPEAKER_01:  911, what's the address of your emergency?\n 0015.6s 0019.3s SPEAKER_00:  Hi, we're located, I forgot the address here, what's that?\n 0020.6s 0023.9s SPEAKER_00:  3971 Bradford Court.\n 0024.5s 0026.3s SPEAKER_01:  Okay, and this is the Abington about.\n 0026.6s 0031.4s SPEAKER_00:  It is, and we are smelling something burning and we're seeing smoke.\n 0033.1s 0034.3s SPEAKER_01:  Okay, is he smoked?\n 0034.6s 0035.0s SPEAKER_00:  Yes.\n 0035.3s 0047.0s SPEAKER_01:  Okay, well I'll just advise you, we can start the Fire Department that way. If you guys have any evacuation procedures, just use the Metro on policy or discretion. In the meantime, can I get your name?\n 0047.4s 0048.3s SPEAKER_00:  My name is Julie.\n 0049.1s 0051.8s SPEAKER_01:  And Julie, what was your phone number in case we get disconnected?\n 0053.2s 0053.7s SPEAKER_00:  Um...\n 0054.5s 0057.1s SPEAKER_00:  I don't have the number right off.\n 0057.3s 0059.0s SPEAKER_01:  yourself on the

In [ ]:
def process_text(text):
    text = str(text).lower()
#     text = re.sub(
#         f"[{re.escape(string.punctuation)}]", " ", text
#     )
    text = " ".join(text.split())
    return text

df["clean_text"] = df.text.map(process_text)

In [ ]:
def unicodes_remove(text):
    text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
    return text
df["clean_text"] = df.clean_text.map(unicodes_remove)


In [ ]:
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
def remove_stopwords(text):
    text = " ".join([word for word in text.split() if word not in (stop)])
    return text
df["clean_text"] = df.clean_text.map(remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def chemical_keywords_extract(text): #extract chemical keywords
  d=''
  for sent in [line.strip() for line in text.split(" ")]:
    doc = model(sent)
    for ent in doc.ents:
      if ent.label_=='CHEMICAL':
        d=d+(ent.text)
        d=d+' '
  d=d+'No'
  return (d)
df["key_words_chemical"] = df.clean_text.map(chemical_keywords_extract)

In [ ]:
df


,text,target,clean_text,key_words_chemical
0,"0012.5s 0014.7s SPEAKER_01: 911, what's the a...",fire-commercial,00125s 00147s speaker01 911 whats address emer...,smelling smoke smoke smoke smoke smoke No
1,"0011.1s 0012.9s SPEAKER_01: 911, let's addres...",sick person,00111s 00129s speaker01 911 lets address emerg...,00337s shoot 01167s 01167s No
2,"0011.0s 0014.2s SPEAKER_00: 911, what's he ad...",lift assist,00110s 00142s speaker00 911 whats address emer...,No
3,0004.8s 0006.0s SPEAKER_00: Don't forget to t...,injury (basic),00048s 00060s speaker00 dont forget talk 00124...,No
4,"0010.6s 0012.4s SPEAKER_00: 911, what's the a...",sick person,00106s 00124s speaker00 911 whats address miss...,No
...,...,...,...,...
725,"0011.4s 0013.8s SPEAKER_02: 911, what is the ...",911 cellular hang-up/open/misdial,00114s 00138s speaker02 911 address emergency ...,00351s No
726,"\n 0014.1s 0018.3s SPEAKER_01: 911, what's th...",fire - lightning no smoke/fire,00141s 00183s speaker01 911 whats address emer...,smoke No
727,0002.0s 0002.7s SPEAKER_00: What's your phone...,missing person,00020s 00027s speaker00 whats phone number 000...,No
728,"0011.4s 0013.8s SPEAKER_02: 911, what is the ...",911 cellular hang-up/open/misdial,00114s 00138s speaker02 911 address emergency ...,00351s No


In [ ]:
def disease_keywords_extract(text): #disease
  d=''
  counter = 0
  for sent in [line.strip() for line in text.split(" ")]:
    doc = model(sent)
    for ent in doc.ents:
      if ent.label_=='DISEASE':
        d=d+(ent.text)
        d=d+' '
    counter += 1
    print(f"Processed {counter} out of {len(text.split())} sentences")
  d=d+'No'
  return (d)

df["key_words_disease"] = df.clean_text.map(disease_keywords_extract)


Streaming output truncated to the last 5000 lines.
Processed 480 out of 531 sentences
Processed 481 out of 531 sentences
Processed 482 out of 531 sentences
Processed 483 out of 531 sentences
Processed 484 out of 531 sentences
Processed 485 out of 531 sentences
Processed 486 out of 531 sentences
Processed 487 out of 531 sentences
Processed 488 out of 531 sentences
Processed 489 out of 531 sentences
Processed 490 out of 531 sentences
Processed 491 out of 531 sentences
Processed 492 out of 531 sentences
Processed 493 out of 531 sentences
Processed 494 out of 531 sentences
Processed 495 out of 531 sentences
Processed 496 out of 531 sentences
Processed 497 out of 531 sentences
Processed 498 out of 531 sentences
Processed 499 out of 531 sentences
Processed 500 out of 531 sentences
Processed 501 out of 531 sentences
Processed 502 out of 531 sentences
Processed 503 out of 531 sentences
Processed 504 out of 531 sentences
Processed 505 out of 531 sentences
Processed 506 out of 531 sentences
Proc

In [60]:
df

,text,target,clean_text,key_words_chemical,key_words_disease
0,"0012.5s 0014.7s SPEAKER_01: 911, what's the a...",fire-commercial,00125s 00147s speaker01 911 whats address emer...,smelling smoke smoke smoke smoke smoke No,No
1,"0011.1s 0012.9s SPEAKER_01: 911, let's addres...",sick person,00111s 00129s speaker01 911 lets address emerg...,00337s shoot 01167s 01167s No,pain stomachache bleeding bleeding bleeding bl...
2,"0011.0s 0014.2s SPEAKER_00: 911, what's he ad...",lift assist,00110s 00142s speaker00 911 whats address emer...,No,stroke No
3,0004.8s 0006.0s SPEAKER_00: Don't forget to t...,injury (basic),00048s 00060s speaker00 dont forget talk 00124...,No,No
4,"0010.6s 0012.4s SPEAKER_00: 911, what's the a...",sick person,00106s 00124s speaker00 911 whats address miss...,No,No
...,...,...,...,...,...
725,"0011.4s 0013.8s SPEAKER_02: 911, what is the ...",911 cellular hang-up/open/misdial,00114s 00138s speaker02 911 address emergency ...,00351s No,No
726,"\n 0014.1s 0018.3s SPEAKER_01: 911, what's th...",fire - lightning no smoke/fire,00141s 00183s speaker01 911 whats address emer...,smoke No,No
727,0002.0s 0002.7s SPEAKER_00: What's your phone...,missing person,00020s 00027s speaker00 whats phone number 000...,No,No
728,"0011.4s 0013.8s SPEAKER_02: 911, what is the ...",911 cellular hang-up/open/misdial,00114s 00138s speaker02 911 address emergency ...,00351s No,No


In [ ]:
 df_train, df_test = train_test_split(df, test_size=0.20, stratify=df.target)  

ValueError: ignored

In [61]:
from sklearn.model_selection import train_test_split

# Separate out instances with only 1 count of target
df_train_1 = df[df['target'].map(df['target'].value_counts()) == 1]
df_train = df[df['target'].map(df['target'].value_counts()) > 1]

# Split remaining instances into train and test sets
df_train, df_test = train_test_split(df_train, test_size=0.20, stratify=df_train.target)

# Combine instances with 1 count of target to the training set
df_train = pd.concat([df_train, df_train_1])



In [62]:
vec = CountVectorizer(
    ngram_range=(1, 3), 
    stop_words="english",
)

In [63]:
from scipy.sparse import coo_matrix, hstack     #transform
X1 = vec.fit_transform(df_train.clean_text)
X_test1 = vec.transform(df_test.clean_text)
X2 = vec.fit_transform(df_train.key_words_disease)
X_test2 = vec.transform(df_test.key_words_disease)
X_train = hstack([X1,X2])
# X_test = vec.transform(df_test.clean_text)
# X_test2 = vec.transform(df_test.key_words_disease)
X_test = hstack([X_test1,X_test2])

y_train = df_train.target
y_test = df_test.target

In [ ]:
print(df_test['target'].value_counts())


sick person                          17
injured from a fall                  15
lift assist                           9
difficulty breathing                  8
chest pain                            8
911 cellular hang-up/open/misdial     6
domestic                              6
assist other agency/unit              5
mental health crisis                  4
suicide attempt                       3
diabetic problem                      3
unresponsive / unconscious            3
911 hang-up/open/misdial              3
co detector no medical                2
back pain                             2
injured person - basic                2
stroke                                2
service run - fire                    2
suspicious person                     2
overdose / poisoning                  2
heart problems                        2
suspicious person/vehicle             2
seizure                               2
cardiac or respiratory arrest         1
allergic reaction                     1


In [64]:
from sklearn import svm #svm model 
clf = svm.SVC()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print(classification_report(y_test, preds))

                                   precision    recall  f1-score   support

911 cellular hang-up/open/misdial       0.30      0.50      0.37         6
         911 hang-up/open/misdial       1.00      0.33      0.50         3
                   abdominal pain       0.00      0.00      0.00         1
                allergic reaction       0.00      0.00      0.00         1
                      animal call       0.00      0.00      0.00         1
         assist other agency/unit       0.00      0.00      0.00         5
           auto accident injuries       0.00      0.00      0.00         1
         auto accident non injury       0.00      0.00      0.00         1
         auto accident-non injury       0.00      0.00      0.00         1
                        back pain       0.00      0.00      0.00         3
             burglary in progress       0.00      0.00      0.00         1
     cardiac / respiratory arrest       0.00      0.00      0.00         1
    cardiac or respirato

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [65]:
preds

array(['911 cellular hang-up/open/misdial', 'sick person',
       'injured from a fall', 'sick person', 'sick person', 'sick person',
       'sick person', 'sick person', 'sick person', 'sick person',
       'injured from a fall', '911 cellular hang-up/open/misdial',
       'injured from a fall', '911 cellular hang-up/open/misdial',
       'sick person', 'sick person', 'injured from a fall', 'sick person',
       'sick person', 'sick person', 'sick person',
       '911 cellular hang-up/open/misdial', 'sick person', 'sick person',
       'sick person', 'sick person', 'chest pain', 'sick person',
       'sick person', 'injured from a fall', 'sick person', 'sick person',
       'injured from a fall', 'injured from a fall', 'lift assist',
       'sick person', 'sick person', 'sick person', 'sick person',
       'sick person', 'injured from a fall', 'injured from a fall',
       'sick person', 'sick person', 'sick person', 'sick person',
       'sick person', 'injured from a fall', 'chest p

In [66]:
y_test

682    911 cellular hang-up/open/misdial
446         cardiac / respiratory arrest
139                 difficulty breathing
421                          sick person
274                            back pain
                     ...                
103                          sick person
627                       duplicate call
423                          lift assist
104                          sick person
140                           chest pain
Name: target, Length: 138, dtype: object

In [67]:
nb = MultinomialNB()  #second model
nb.fit(X_train, y_train)

preds = nb.predict(X_test)
print(classification_report(y_test, preds))

                                   precision    recall  f1-score   support

911 cellular hang-up/open/misdial       1.00      0.17      0.29         6
         911 hang-up/open/misdial       1.00      0.33      0.50         3
                   abdominal pain       0.00      0.00      0.00         1
                allergic reaction       0.00      0.00      0.00         1
                      animal call       0.00      0.00      0.00         1
         assist other agency/unit       0.50      0.20      0.29         5
           auto accident injuries       0.00      0.00      0.00         1
         auto accident non injury       0.00      0.00      0.00         1
         auto accident-non injury       0.00      0.00      0.00         1
                        back pain       0.00      0.00      0.00         3
             burglary in progress       0.00      0.00      0.00         1
     cardiac / respiratory arrest       0.00      0.00      0.00         1
    cardiac or respirato

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
